In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Date, DateTime, Float, BigInteger, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
engine = create_engine("sqlite://", echo=True)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

/var/folders/hq/659b4zd16vscxjxb3lxdr18c0000gp/T/ipykernel_2374/1111253954.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [4]:
class Person(Base):
    __tablename__ = 'PERSON'
    person_id = Column(Integer, primary_key=True, nullable=False)
    gender_concept_id = Column(Integer, nullable=False)
    year_of_birth = Column(Integer, nullable=False)
    month_of_birth = Column(Integer)
    day_of_birth = Column(Integer)
    birth_datetime = Column(DateTime)
    race_concept_id = Column(Integer, nullable=False)
    ethnicity_concept_id = Column(Integer, nullable=False)
    location_id = Column(Integer)
    provider_id = Column(Integer)
    care_site_id = Column(Integer)
    person_source_value = Column(String(50))
    gender_source_value = Column(String(50))
    gender_source_concept_id = Column(Integer)
    race_source_value = Column(String(50))
    race_source_concept_id = Column(Integer)
    ethnicity_source_value = Column(String(50))
    ethnicity_source_concept_id = Column(Integer)

class ObservationPeriod(Base):
    __tablename__ = 'OBSERVATION_PERIOD'
    observation_period_id = Column(Integer, primary_key=True, nullable=False)
    person_id = Column(Integer, nullable=False)
    observation_period_start_date = Column(Date, nullable=False)
    observation_period_end_date = Column(Date, nullable=False)
    period_type_concept_id = Column(Integer, nullable=False)

# Add more class definitions for other tables following the same pattern

# Create the tables in the database
Base.metadata.create_all(engine)


2024-04-06 10:26:41,896 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 10:26:41,897 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("PERSON")
2024-04-06 10:26:41,897 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 10:26:41,900 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("PERSON")
2024-04-06 10:26:41,901 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 10:26:41,902 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("OBSERVATION_PERIOD")
2024-04-06 10:26:41,903 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 10:26:41,904 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("OBSERVATION_PERIOD")
2024-04-06 10:26:41,904 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 10:26:41,906 INFO sqlalchemy.engine.Engine 
CREATE TABLE "PERSON" (
	person_id INTEGER NOT NULL, 
	gender_concept_id INTEGER NOT NULL, 
	year_of_birth INTEGER NOT NULL, 
	month_of_birth INTEGER, 
	day_of_birth INTEGER, 
	birth_datetime DATETIME, 
	race_concept_id INTEGER NOT NU

In [13]:
from sqlalchemy import create_engine, text

# Connection details - replace with your actual details
DATABASE = 'test'  # The name of your MySQL database
USERNAME = 'root'  # Your MySQL database username
PASSWORD = 'TSUtigers123'  # Your MySQL database password
SERVER = 'localhost'  # The hostname or IP address of your MySQL server

# SQLAlchemy connection string for MySQL using mysqlconnector
connection_string = f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}'

# Creating the engine for MySQL
engine = create_engine(connection_string, echo=True)

# Path to your SQL file
sql_file_path = './OMOPCDM_sql_server_5.4_ddl.sql'  # Make sure the SQL commands are compatible with MySQL

# Open, read, and prepare the SQL file
with open(sql_file_path, 'r') as sql_file:
    sql_commands = sql_file.read()

    # Since you're working with MySQL, ensure SQL commands in your file are compatible with MySQL
    # Adjust any placeholder replacements as necessary, depending on your SQL file's content

# Execute the SQL commands on MySQL
with engine.begin() as conn:
    # Assuming your SQL commands are separated by semicolons, execute them individually
    # This approach might need adjustments based on the actual content of your SQL file
    for command in sql_commands.split(';'):
        if command.strip():  # Avoid executing empty commands
            conn.execute(text(command))


2024-04-06 12:41:32,881 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-06 12:41:32,881 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-06 12:41:32,884 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-06 12:41:32,885 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-06 12:41:32,885 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-06 12:41:32,885 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-06 12:41:32,886 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 12:41:32,888 INFO sqlalchemy.engine.Engine --sql server CDM DDL Specification for OMOP Common Data Model 5.4

--HINT DISTRIBUTE ON KEY (person_id)
CREATE TABLE @cdmDatabaseSchema.PERSON (
			person_id integer NOT NULL,
			gender_concept_id integer NOT NULL,
			year_of_birth integer NOT NULL,
			month_of_birth integer NULL,
			day_of_birth integer NULL,
			birth_datetime datetime NULL,
			race_concept_id integer NOT NULL,
			ethnicity_concept_id integer NOT NULL,
			location_id integ

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '--sql server CDM DDL Specification for OMOP Common Data Model 5.4

--HINT DISTRI' at line 1
[SQL: --sql server CDM DDL Specification for OMOP Common Data Model 5.4

--HINT DISTRIBUTE ON KEY (person_id)
CREATE TABLE @cdmDatabaseSchema.PERSON (
			person_id integer NOT NULL,
			gender_concept_id integer NOT NULL,
			year_of_birth integer NOT NULL,
			month_of_birth integer NULL,
			day_of_birth integer NULL,
			birth_datetime datetime NULL,
			race_concept_id integer NOT NULL,
			ethnicity_concept_id integer NOT NULL,
			location_id integer NULL,
			provider_id integer NULL,
			care_site_id integer NULL,
			person_source_value varchar(50) NULL,
			gender_source_value varchar(50) NULL,
			gender_source_concept_id integer NULL,
			race_source_value varchar(50) NULL,
			race_source_con

In [ ]:
from sqlalchemy import create_engine, text

# Assuming USERNAME, PASSWORD, SERVER, and DATABASE variables are defined above this line
connection_string = f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}'
engine = create_engine(connection_string, echo=True)

# Path to your SQL file
sql_file_path = './OMOPCDM_sql_server_5.4_ddl.sql'

# Initialize an empty list to hold executable lines
executable_lines = []

with open(sql_file_path, 'r') as sql_file:
    for line in sql_file:
        # Remove the placeholder from each line if it exists
        line = line.replace('@cdmDatabaseSchema.', '')

        # Filter out lines that are comments or empty
        if not line.strip().startswith('--') and line.strip():
            executable_lines.append(line)

# Combine filtered and placeholder-removed lines back into commands, split by semicolon
commands = "".join(executable_lines).split(';')

# Execute commands
with engine.begin() as conn:
    for command in commands:
        command = command.strip()  # Avoid executing empty commands
        if command:  # Check if command is not empty
            conn.execute(text(command))


In [38]:
sql_file_path = './OMOPCDM_sql_server_5.4_primary_keys.sql'

# Read SQL commands from the file and replace the placeholder
with open(sql_file_path, 'r') as file:
    sql_commands = file.read().replace('@cdmDatabaseSchema.', '')

# Split the modified SQL commands into individual commands
# Assuming each command ends with a ';' and possibly a newline for readability
commands = sql_commands.split(';\n')

# Execute each command
with engine.begin() as connection:
    for command in commands:
        command = command.strip()  # Clean whitespace
        if command:  # Check if command is not empty
            connection.execute(text(command))

2024-04-06 13:14:43,485 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 13:14:43,486 INFO sqlalchemy.engine.Engine ALTER TABLE PERSON ADD CONSTRAINT xpk_PERSON PRIMARY KEY NONCLUSTERED (person_id)
2024-04-06 13:14:43,487 INFO sqlalchemy.engine.Engine [generated in 0.00086s] {}
2024-04-06 13:14:43,561 INFO sqlalchemy.engine.Engine ALTER TABLE OBSERVATION_PERIOD ADD CONSTRAINT xpk_OBSERVATION_PERIOD PRIMARY KEY NONCLUSTERED (observation_period_id)
2024-04-06 13:14:43,561 INFO sqlalchemy.engine.Engine [generated in 0.00068s] {}
2024-04-06 13:14:43,574 INFO sqlalchemy.engine.Engine ALTER TABLE VISIT_OCCURRENCE ADD CONSTRAINT xpk_VISIT_OCCURRENCE PRIMARY KEY NONCLUSTERED (visit_occurrence_id)
2024-04-06 13:14:43,575 INFO sqlalchemy.engine.Engine [generated in 0.00081s] {}
2024-04-06 13:14:43,585 INFO sqlalchemy.engine.Engine ALTER TABLE VISIT_DETAIL ADD CONSTRAINT xpk_VISIT_DETAIL PRIMARY KEY NONCLUSTERED (visit_detail_id)
2024-04-06 13:14:43,585 INFO sqlalchemy.engine.Engine [gen

In [36]:
from sqlalchemy.engine.reflection import Inspector

inspector = Inspector.from_engine(engine)

# Retrieve and print all table names using the Inspector
table_names = inspector.get_table_names()
print("Tables in the database:")
for table_name in table_names:
    print(table_name)

2024-04-06 13:08:12,566 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 13:08:12,567 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `test`
2024-04-06 13:08:12,567 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-06 13:08:12,601 INFO sqlalchemy.engine.Engine ROLLBACK
Tables in the database:
CARE_SITE
CDM_SOURCE
COHORT
COHORT_DEFINITION
CONCEPT
CONCEPT_ANCESTOR
CONCEPT_CLASS
CONCEPT_RELATIONSHIP
CONCEPT_SYNONYM
CONDITION_ERA
CONDITION_OCCURRENCE
COST
DEATH
DEVICE_EXPOSURE
DOMAIN
DOSE_ERA
DRUG_ERA
DRUG_EXPOSURE
DRUG_STRENGTH
EPISODE
EPISODE_EVENT
FACT_RELATIONSHIP
LOCATION
MEASUREMENT
METADATA
NOTE
NOTE_NLP
OBSERVATION
OBSERVATION_PERIOD
PAYER_PLAN_PERIOD
PERSON
PROCEDURE_OCCURRENCE
PROVIDER
RELATIONSHIP
SOURCE_TO_CONCEPT_MAP
SPECIMEN
VISIT_DETAIL
VISIT_OCCURRENCE
VOCABULARY


/var/folders/hq/659b4zd16vscxjxb3lxdr18c0000gp/T/ipykernel_1688/1350057838.py:3: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = Inspector.from_engine(engine)


In [39]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.schema import CreateTable
from sqlalchemy.engine.reflection import Inspector

# Assuming USERNAME, PASSWORD, SERVER, and DATABASE variables are defined above this line
connection_string = f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}'
engine = create_engine(connection_string, echo=True)

# Your existing code to execute SQL commands from a file...

# Now, let's print the DDL for each table
inspector = Inspector.from_engine(engine)

# Reflect the database schema into a new MetaData instance
metadata = MetaData()
metadata.reflect(bind=engine)

print("DDL for tables in the database:")
for table_name in inspector.get_table_names():
    # Using the reflected metadata to get the table object
    table = metadata.tables[table_name]
    # Generating the CREATE TABLE statement for the table
    ddl = CreateTable(table).compile(engine).string
    print(f"\nDDL for {table_name}:\n{ddl}")


2024-04-06 13:15:10,704 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-06 13:15:10,705 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-06 13:15:10,708 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-06 13:15:10,708 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-06 13:15:10,709 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-06 13:15:10,709 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-06 13:15:10,710 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 13:15:10,710 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `test`
2024-04-06 13:15:10,711 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-06 13:15:10,722 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `CARE_SITE`
2024-04-06 13:15:10,722 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-06 13:15:10,725 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `CDM_SOURCE`
2024-04-06 13:15:10,725 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-06 13:15:10,727 INFO sqlalchemy.engin

/var/folders/hq/659b4zd16vscxjxb3lxdr18c0000gp/T/ipykernel_1688/3633835634.py:12: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = Inspector.from_engine(engine)


In [41]:
import re

# Path to your SQL file containing CREATE INDEX statements
sql_file_path = './OMOPCDM_sql_server_5.4_indices.sql'

# Read SQL commands from the file and replace the placeholder
with open(sql_file_path, 'r') as file:
    # Read the entire file content
    sql_commands = file.read()
# Remove multiline comments
sql_commands = re.sub(r'/\*.*?\*/', '', sql_commands, flags=re.DOTALL)

# Replace the placeholder
sql_commands = sql_commands.replace('@cdmDatabaseSchema.', '')

# Split the modified SQL commands into individual commands
commands = sql_commands.split(';')

# Filter out empty commands and commands that are commented out
filtered_commands = []
for command in commands:
    # Trim whitespace and remove single-line comments
    stripped_command = command.strip()
    if stripped_command and not stripped_command.startswith('--'):
        filtered_commands.append(stripped_command)

# Execute each filtered command
with engine.begin() as connection:
    for command in filtered_commands:
        try:
            connection.execute(text(command))
        except Exception as e:
            print(f"An error occurred: {e}")
            print(f"While executing: {command}\n")

print("Index creation commands executed successfully.")

2024-04-06 13:35:19,962 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 13:35:19,990 INFO sqlalchemy.engine.Engine CREATE CLUSTERED INDEX idx_person_id ON person (person_id ASC)
2024-04-06 13:35:19,991 INFO sqlalchemy.engine.Engine [generated in 0.02406s] {}
An error occurred: (mysql.connector.errors.ProgrammingError) 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'CLUSTERED INDEX idx_person_id ON person (person_id ASC)' at line 1
[SQL: CREATE CLUSTERED INDEX idx_person_id ON person (person_id ASC)]
(Background on this error at: https://sqlalche.me/e/20/f405)
While executing: CREATE CLUSTERED INDEX idx_person_id ON person (person_id ASC)

2024-04-06 13:35:19,998 INFO sqlalchemy.engine.Engine CREATE INDEX idx_gender ON person (gender_concept_id ASC)
2024-04-06 13:35:19,999 INFO sqlalchemy.engine.Engine [generated in 0.00054s] {}
2024-04-06 13:35:20,035 INFO sqlalchemy.engine.En